In [ ]:
ROOT_DIR = "/kaggle/working"
GIT_CHECKOUT_ARG = "refactor-model-module"

In [ ]:
# Clone repo and download dataset

import os

if not os.path.exists(f"{ROOT_DIR}/neural-swipe-typing"):
    !git clone https://github.com/proshian/neural-swipe-typing.git {ROOT_DIR}/neural-swipe-typing
    %cd {ROOT_DIR}/neural-swipe-typing
    !git checkout {GIT_CHECKOUT_ARG}

if not os.path.exists(f"{ROOT_DIR}/neural-swipe-typing/data/data_preprocessed/valid.jsonl"):
    %cd {ROOT_DIR}/neural-swipe-typing/src
    !python ./data_obtaining_and_preprocessing/download_dataset_preprocessed.py
    %cd {ROOT_DIR}/neural-swipe-typing

In [ ]:
%cd {ROOT_DIR}/neural-swipe-typing

In [ ]:
!git checkout {GIT_CHECKOUT_ARG}
!git pull

In [ ]:
# !!! Note: package version differ from {ROOT_DIR}/neural-swipe-typing/requirements/requirements.txt
!pip install -q lightning

In [ ]:
!ls  {ROOT_DIR}/neural-swipe-typing/checkpoints/*/epoch_end  

In [ ]:
# If you want to continue from a checkpoint, from your previous run choose from the list above
CKPT_OF_CHOICE = None
EXPERIMENT_NAME = "traj_and_nearest"  # See options in `model_configs_dict`

In [ ]:
import json
import os

In [ ]:
model_configs_dict = {
    "traj_and_nearest": {
        "swipe_feature_extractor_factory_config_path": "./configs/feature_extractor/traj_and_nearest.json",
        "swipe_point_embedder_config_path": "./configs/swipe_point_embedder/separate_traj_and_nearest__6_coord.json",
    },
    "traj_and_weighted": {
        "swipe_feature_extractor_factory_config_path": "./configs/feature_extractor/traj_and_weighted.json",
        "swipe_point_embedder_config_path": "./configs/swipe_point_embedder/separate_traj_and_weighted__6_coord.json",
    },
    "indiswipe": {
        "swipe_feature_extractor_factory_config_path": "./configs/feature_extractor/indiswipe.json",
        "swipe_point_embedder_config_path": "./configs/swipe_point_embedder/separate_traj_and_nearest__4_coord.json",
    },
    "nearest_only": {
        "swipe_feature_extractor_factory_config_path": "./configs/feature_extractor/nearest_key.json",
        "swipe_point_embedder_config_path": "./configs/swipe_point_embedder/nearest.json",
    },
    "google_2015": {
        "swipe_feature_extractor_factory_config_path": "./configs/feature_extractor/google_2015.json",
        "swipe_point_embedder_config_path": "./configs/swipe_point_embedder/separate_traj_and_nearest__3_coord.json",
    },
}

for experiment_name, config in model_configs_dict.items():
    config["experiment_name"] = experiment_name

In [ ]:
model_config = model_configs_dict[EXPERIMENT_NAME]

In [ ]:
TRAIN_CONFIG_WITHOUT_MODEL = {
    "num_classes": 35,
    "max_out_seq_len": 35,
    "grid_name": "default",
    "grids_path": "./data/data_preprocessed/gridname_to_grid.json",
    "trajectory_features_statistics_path": "./data/data_preprocessed/trajectory_features_statistics.json",
    "bounding_boxes_path": "./data/data_preprocessed/key_bounding_boxes.json",
    "keyboard_tokenizer_path": "./tokenizers/keyboard/ru.json",
    "dataset_paths": {
        # "train": "./data/data_preprocessed/train_filtered.jsonl",
        "train": "./data/data_preprocessed/valid.jsonl",
        "val": "./data/data_preprocessed/valid.jsonl"
    },
    "dataloader_num_workers": 4,
    "train_batch_size": 256,
    "val_batch_size": 512,
    "vocab_path": "./data/data_preprocessed/voc.txt",
    "train_total": 5591814,
    "val_total": 10000,
    "seed": 42,
    "early_stopping": {
        "enabled": False,
        "patience": 30
    },
    "lr_scheduler": {
        "type": "ReduceLROnPlateau",
        "params": {
            "factor": 0.5,
            "patience": 8
        }
    },
    "path_to_continue_checkpoint": f"{ROOT_DIR}/neural-swipe-typing/checkpoints/epoch_end/{CKPT_OF_CHOICE}" if CKPT_OF_CHOICE else None,
    "model_name": "v3_nearest_and_traj_transformer_bigger",
    "label_smoothing": 0.045,
    "optimizer": {
        "type": "Adam",
        "params": {"lr":1e-4, "weight_decay":0}
    },
    # "val_check_interval": 3000,
    "val_check_interval": 10,
    "device": "cuda"
}


train_config = TRAIN_CONFIG_WITHOUT_MODEL | model_config


with open(f'{ROOT_DIR}/neural-swipe-typing/configs/train.json', 'w') as f:
    json.dump(train_config, f)

In [ ]:
# feature_extractor_config = [
#     {
#         "type": "trajectory",
#         "params": {
#             "include_dt": False,
#             "include_velocities": True,
#             "include_accelerations": True
#         }
#     },
#     {
#         "type": "nearest_key",
#         "params": {
#             "use_lookup": True
#         }
#     }
# ]

# sfe_path = os.path.join(
#     f'{ROOT_DIR}/neural-swipe-typing/', 
#     train_config["swipe_feature_extractor_factory_config_path"])

# with open(sfe_path, 'w') as f:
#     json.dump({}, f)

In [ ]:
!python -m src.train --train_config configs/train.json